In [1]:
import pandas as pd
import numpy as np
import string

In [3]:
lines= pd.read_table('english to french/english to french.txt', names=['eng', 'fr'])

In [4]:
lines = lines[0:50000]
print(lines.head(3))
print(lines.tail(3))

    eng        fr
0   Go.      Va !
1  Run!   Cours !
2  Run!  Courez !
                            eng                              fr
49997  They go to work on foot.     Ils vont au travail à pied.
49998  They got into the train.    Ils montèrent dans le train.
49999  They got into the train.  Elles montèrent dans le train.


In [5]:
lines['eng']=lines['eng'].apply(lambda x: x.lower())
lines['fr']=lines['fr'].apply(lambda x: x.lower())

In [6]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fr=lines.fr.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [7]:
print(lines.head(3))
print(lines.tail(3))

   eng       fr
0   go      va 
1  run   cours 
2  run  courez 
                           eng                             fr
49997  they go to work on foot     ils vont au travail à pied
49998  they got into the train    ils montèrent dans le train
49999  they got into the train  elles montèrent dans le train


In [8]:
lines.fr = lines.fr.apply(lambda x : 'start '+ x + ' end')

In [9]:
print(lines.head(3))
print(lines.tail(3))

   eng                 fr
0   go      start va  end
1  run   start cours  end
2  run  start courez  end
                           eng                                       fr
49997  they go to work on foot     start ils vont au travail à pied end
49998  they got into the train    start ils montèrent dans le train end
49999  they got into the train  start elles montèrent dans le train end


In [10]:
lines.shape

(50000, 2)

In [11]:
# fit a tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [12]:
import json
eng_tokenizer = create_tokenizer(lines['eng'])
eng_dict=json.loads(json.dumps(eng_tokenizer.word_counts))
df =pd.DataFrame([eng_dict.keys(), eng_dict.values()]).T
df.columns = ['word','count']

df = df.sort_values(by='count',ascending = False)
df['cum_count']=df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_eng_words = df[df['cum_perc']<0.8]['word'].values
#final_eng_words=df['word'].values

In [13]:
fr_tokenizer = create_tokenizer(lines['fr'])
fr_dict = json.loads(json.dumps(fr_tokenizer.word_counts))
df =pd.DataFrame([fr_dict.keys(), fr_dict.values()]).T
df.columns = ['word','count']
df = df.sort_values(by='count',ascending = False)
df['cum_count']=df['count'].cumsum()
df['cum_perc'] = df['cum_count']/df['cum_count'].max()
final_fr_words = df[df['cum_perc']<0.8]['word'].values
#final_fr_words=df['word'].values

In [14]:
print(len(final_eng_words),len(final_fr_words))

384 357


In [15]:
def filter_eng_words(x):
  t = []
  x = x.split()
  for i in range(len(x)):
    if x[i] in final_eng_words:
      t.append(x[i])
    else:
      t.append('unk')
  x3 = ''
  for i in range(len(t)):
    x3 = x3+t[i]+' '
  return x3

In [16]:
filter_eng_words('he is extremely good')

'he is unk good '

In [17]:
def filter_fr_words(x):
  t = []
  x = x.split()
  for i in range(len(x)):
    if x[i] in final_fr_words:
      t.append(x[i])
    else:
      t.append('unk')
  x3 = ''
  for i in range(len(t)):
    x3 = x3+t[i]+' '
  return x3

In [18]:
lines['eng']=lines['eng'].apply(filter_eng_words)
lines['fr']=lines['fr'].apply(filter_fr_words)

In [19]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_french_words=set()
for fr in lines.fr:
    for word in fr.split():
        if word not in all_french_words:
            all_french_words.add(word)

In [20]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_french_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_french_words)
# del all_eng_words, all_french_words

In [21]:
set(all_french_words) - set(final_fr_words) 

{'unk'}

In [22]:
len(all_eng_words)

385

In [23]:
len(target_words)

358

In [24]:
input_token_index = dict(
    [(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i+1) for i, word in enumerate(target_words)])

In [25]:
num_decoder_tokens

358

In [26]:
print(target_token_index['start']) 
print(target_token_index['end']) 
print(list(input_token_index.keys())[384])
print(list(target_token_index.keys())[283])
print(list(target_token_index.keys())[88])

284
89
youve
start
end


In [27]:
length_list=[]
for l in lines.fr:
    length_list.append(len(l.split(' ')))
fr_max_length = np.max(length_list)

In [28]:
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
eng_max_length = np.max(length_list)

In [29]:
print(eng_max_length)
print(fr_max_length)

8
17


In [30]:
encoder_input_data = np.zeros(
    (len(lines['eng']), eng_max_length),
    dtype='float32')
decoder= np.zeros(
    (len(lines['fr']), fr_max_length+1),
    dtype='float32')


In [31]:
decoder.shape

(50000, 18)

In [32]:
for i, (input_text, target_text) in enumerate(zip(lines['eng'], lines['fr'])):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        decoder[i, t] = target_token_index[word]
decoder_input_data=decoder[:,:-1]
decoder_target_data=decoder[:,1:]

In [33]:
print(encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape)

(50000, 8) (50000, 17) (50000, 17)


In [34]:
print(encoder_input_data[1000])
print(decoder_input_data[1000])
print(decoder_target_data[1000])

[63.  8.  0.  0.  0.  0.  0.  0.]
[284. 320. 274.  89.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.]
[320. 274.  89.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.]


In [35]:
import tensorflow as tf
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [36]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)#(1,position,d_model)


In [37]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  #x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0]
  # add extra dimensions to add the padding
  # to the attention logits.
  return mask[:, tf.newaxis, tf.newaxis, :]  # (batch, 1, 1, seq_len)

In [38]:
def create_look_ahead_mask(x):
    seq_len=tf.shape(x)[1]
    look_ahead_mask=1-tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)#lower,upper
    padding_mask=create_padding_mask(x)
    return tf.maximum(look_ahead_mask,padding_mask)

##x:tokenized words with padding for each sentence 1D텐서 [...,0,0,0](ts,), return (ts,ts)

In [39]:
def scaled_dot_product_attention(q, k, v, mask):
 

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k) (number of heads,ts,ts)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v) (number of heads,ts,64)

  return output, attention_weights

In [40]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads  #8
    self.d_model = d_model #128

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model) 
    self.wk = tf.keras.layers.Dense(d_model) 
    self.wv = tf.keras.layers.Dense(d_model) 

    self.dense = tf.keras.layers.Dense(d_model) # scale_dot_attention후 concatenate한 attention (ts,128)를 Dense에 통과하기 위함

  def split_heads(self, x, batch_size):
   
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  
    k = self.wk(k) 
    v = self.wv(v) 

    q = self.split_heads(q, batch_size) 
    k = self.split_heads(k, batch_size)  
    v = self.split_heads(v, batch_size) 

    
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [41]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff) seq_len=ts
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model) d_model=128
  ])

In [42]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model) input_seq_len=input_ts, d_model=128
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [43]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model) x: decoder input

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model) 
    attn1 = self.dropout1(attn1)                   #targer_seq_len=decoder_input_ts
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    attn2 = self.dropout2(attn2)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model) #target_seq_len=decoder_input_ts

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [44]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, mask):

    seq_len = tf.shape(x)[1]  #x는 tokenized sentence with padding

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [45]:
a={}
for i in range(2):
    block1=i**2
    block2=i**3
    a["a_{}".format(i+1)]=block1
    a["b_{}".format(i+1)]=block2
print(a)   

{'a_1': 0, 'b_1': 0, 'a_2': 1, 'b_2': 1}


In [46]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output,look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [47]:
def transformer(num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, ts_input, ts_target, rate=0.1):
    
    enc_input = tf.keras.Input(shape=(None,))  
    dec_input = tf.keras.Input(shape=(None,))
    
    enc_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    look_ahead_mask=tf.keras.layers.Lambda(create_look_ahead_mask,output_shape=(1,None,None))(dec_input)
    dec_padding_mask=tf.keras.layers.Lambda(create_padding_mask,output_shape=(1,1,None))(enc_input)
    
    enc_output=Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, ts_input, rate)(enc_input,enc_padding_mask)
    dec_output,_ =Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, ts_target, rate)(dec_input,enc_output,look_ahead_mask,dec_padding_mask)
    outputs=tf.keras.layers.Dense(target_vocab_size)(dec_output)
    
    return tf.keras.Model(inputs=[enc_input, dec_input], outputs=outputs)
    


In [48]:
import tensorflow as tf
a=[False,False,True]
b=[False,True,True]
c=tf.math.logical_and(a,b)
print(c)

tf.Tensor([False False  True], shape=(3,), dtype=bool)


In [49]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [50]:
def loss_function(y_true, y_pred):
  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  loss_ = loss_object(y_true, y_pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask) #padding을 제외한 loss


def accuracy_function(y_true, y_pred):
  y_hat=tf.cast(tf.argmax(y_pred,axis=2), dtype=tf.float32) #y_true가 float32이기 때문
  accuracies = tf.equal(y_true, y_hat)
  #accuracies = tf.equal(y_true, tf.argmax(y_pred, axis=2)) #True, False로 출력

  mask = tf.math.logical_not(tf.math.equal(y_true, 0))
  accuracies = tf.math.logical_and(mask, accuracies) #둘다 true일 때만 True

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

#def accuracy(y_true,y_pred):
#    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


In [51]:
import tensorflow as tf
data_set=tf.data.Dataset.from_tensor_slices(((encoder_input_data,decoder_input_data),decoder_target_data))
data_final=data_set.shuffle(1000).batch(64).prefetch(64)#memory를 차지하지 않음

In [52]:
model=transformer(num_layers=1,d_model=128,num_heads=4,dff=256, 
                  input_vocab_size=386, target_vocab_size=359,ts_input=8,ts_target=17,rate=0.2)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy_function])
model.fit(data_final,epochs=30)

Epoch 1/30
782/782 [==============================] - 25s 29ms/step - loss: 2.4657 - accuracy_function: 0.5192
Epoch 2/30
782/782 [==============================] - 26s 33ms/step - loss: 1.3062 - accuracy_function: 0.6737
Epoch 3/30
782/782 [==============================] - 26s 33ms/step - loss: 1.1579 - accuracy_function: 0.6992
Epoch 4/30
782/782 [==============================] - 27s 34ms/step - loss: 1.0805 - accuracy_function: 0.7118
Epoch 5/30
782/782 [==============================] - 27s 34ms/step - loss: 1.0394 - accuracy_function: 0.7184
Epoch 6/30
782/782 [==============================] - 31s 40ms/step - loss: 1.0047 - accuracy_function: 0.7242
Epoch 7/30
782/782 [==============================] - 38s 48ms/step - loss: 0.9784 - accuracy_function: 0.7301
Epoch 8/30
782/782 [==============================] - 30s 38ms/step - loss: 0.9533 - accuracy_function: 0.7338
Epoch 9/30
782/782 [==============================] - 26s 34ms/step - loss: 0.9366 - accuracy_function: 0.7364
E